In [9]:
import numpy as np
import tensorflow as tf

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
learning_rate = 0.01
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)


reset_graph()

batch_norm_momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
training = tf.placeholder_with_default(False, shape=(), name='training')
he_init = tf.variance_scaling_initializer()
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
    bn1 = tf.layers.batch_normalization(hidden1, training = training, momentum = 0.9, name= 'batch_1')
    bn1_act = tf.nn.elu(bn1, name = 'activ_hid1')
    hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
    bn2 = tf.layers.batch_normalization(hidden2, training = training, momentum = 0.9, name = "batch_2")
    bn2_act = tf.nn.elu(bn2, name="activ_hid2")
    logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs_before_bn")
    logits = tf.layers.batch_normalization(logits_before_bn, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [10]:
n_epochs = 20
batch_size = 200

In [11]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.892
1 Validation accuracy: 0.916
2 Validation accuracy: 0.9268
3 Validation accuracy: 0.931
4 Validation accuracy: 0.9368
5 Validation accuracy: 0.9394


KeyboardInterrupt: 

In [12]:
[v.name for v in tf.trainable_variables()]

['hidden1/kernel:0',
 'hidden1/bias:0',
 'batch_1/gamma:0',
 'batch_1/beta:0',
 'hidden2/kernel:0',
 'hidden2/bias:0',
 'batch_2/gamma:0',
 'batch_2/beta:0',
 'outputs_before_bn/kernel:0',
 'outputs_before_bn/bias:0',
 'outputs/gamma:0',
 'outputs/beta:0']

In [13]:
[v.name for v in tf.global_variables()]

['hidden1/kernel:0',
 'hidden1/bias:0',
 'batch_1/gamma:0',
 'batch_1/beta:0',
 'batch_1/moving_mean:0',
 'batch_1/moving_variance:0',
 'hidden2/kernel:0',
 'hidden2/bias:0',
 'batch_2/gamma:0',
 'batch_2/beta:0',
 'batch_2/moving_mean:0',
 'batch_2/moving_variance:0',
 'outputs_before_bn/kernel:0',
 'outputs_before_bn/bias:0',
 'outputs/gamma:0',
 'outputs/beta:0',
 'outputs/moving_mean:0',
 'outputs/moving_variance:0']

In [16]:
for op in tf.get_default_graph().get_operations():
    print(op.name)

X
y
training/input
training
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
batch_1/gamma/Initializer/ones
batch_1/gamma
batch_1/gamma/Assign
batch_1/gamma/read
batch_1/beta/Initializer/zeros
batch_1/beta
batch_1/beta/Assign
batch_1/beta/read
batch_1/moving_mean/Initializer/zeros
batch_1/moving_mean
batch_1/moving_mean/Assign
batch_1/moving_mean/read
batch_1/moving_variance/Initializer/ones
batch_1/moving_variance
batch_1/moving_variance/Assign
batch_1/moving_variance/read
dnn/batch_1/moments/mean/reduction_indices
dnn/batch_1/moments

In [17]:
hidden4_out = tf.get_default_graph().get_tensor_by_name("dnn/activ_hid2:0")

In [18]:
hidden4_out

<tf.Tensor 'dnn/activ_hid2:0' shape=(?, 100) dtype=float32>